<a href="https://colab.research.google.com/github/manufabregas/Dataset-Asistencia-a-turnos-m-dicos/blob/main/TP%203/Grupo_2_TP_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Problemática 2: Los/as especialistas pierden una cantidad significativa de dinero cuando un paciente no se presenta a su cita. Esto ocurre porque el consultorio ya ha sido alquilado y la agenda para ese día ya está establecida. Por lo tanto, la ausencia del paciente representa una pérdida tanto de tiempo como de ingresos. Si los especialistas pudieran anticipar qué pacientes tienen una alta probabilidad de no asistir, podrían implementar estrategias para mitigar estas pérdidas. Por ejemplo, podrían enviar notificaciones específicas a estos pacientes para confirmar sus citas, o bien ajustar la agenda para incluir turnos de refuerzo (sobreturno) cerca de los horarios de pacientes con alta probabilidad de ausencia. Esta proactividad permitiría optimizar la gestión del tiempo y los recursos, reduciendo así el impacto económico de las ausencias.**

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
import pandas as pd
import numpy as np

In [5]:
df_base=pd.read_csv("./base_turnos_hospitales.csv")
df_base = df_base[['msp_name','doc_asistance_rate', 'msp_asistance_rate','dni_asistance_rate','age_avg','qt_canc_pac_ac', 'app_days_gap',"attendance"]]
df_base.head()

,msp_name,doc_asistance_rate,msp_asistance_rate,dni_asistance_rate,age_avg,qt_canc_pac_ac,app_days_gap,attendance
0,NEUROLOGIA,72.831978,71.085359,72.727273,77.0,2.0,39,0
1,CARDIOLOGIA INFANTIL,71.698113,75.919732,50.000000,7.0,0.0,48,0
2,NEUROLOGIA,72.831978,71.085359,37.500000,60.0,0.0,33,0
3,GINECOLOGIA Y OBSTETRICIA,89.130435,86.914721,50.000000,34.0,0.0,1,1
4,GINECOLOGIA,76.271186,71.529776,100.000000,60.0,0.0,13,1


Lista de tareas
Definir el problema de investigación a realizar. Si se trata de aprendizaje supervisado deberán elegir si lo abordarán como un problema de clasificación o regresión. Explicar qué motivación les llevó a elegir esta problemática en particular.
Seleccionar y preparar las variables que formarán parte de su modelo. Si consideran que hay otra variable a agregar y no fue construída con anterioridad lo pueden hacer. También variar en la naturaleza de las variables. Por ejemplo, si existe una variable que es continua (alguna tasa de asistencia o edad, etc) es aquí donde la pueden categorizar de la manera que consideren interesante abordar en el modelo.
Construir el modelo. A continuación se dejan algunas observaciones:
Seleccionar el algoritmo que consideren adecuado para el problema planteado y las variables elegidas.
Dividir el conjunto de datos en entrenamiento y prueba, y si lo consideran, en validación. Asegurar su replicabilidad.
Entrenamiento del modelo.
Optimización de hiperparámetros.
En cada punto, justificar brevemente cada elección.
Evaluar el modelo.
Evaluar el rendimiento del modelo a partir de métricas acordes a su problema de investigación. Interpretar.
Realizar validación cruzada.
Análizar y evaluar posibles mejoras del modelo frente a errores. (Explicar por qué se cree que tal algoritmo aplicado no dió los resultados deseados o esperados)
El punto 3 y 4 deberán aplicarse sobre dos algoritmos. La idea es que realicen la construcción y evaluación de dos modelos que consideren, para luego completar la próxima actividad.

Comparación de modelos. A partir de los dos modelos aplicados se deberá:
Mencionar las principales caracteristicas (ventajas y desventajas generales) de cada algoritmo.
Fundamentar la elección de cada uno ante el problema de investigación planteado. ¿Por qué lo consideré en primer momento? ¿Por qué este algoritmo en primera instancia era el que mejores resultados podría arrojar para este conjunto de datos?
Comparar los resultados obtenidos con cada modelo a partir de las métricas, así como tiempos y costo computacional que insumieron.
Elegir un modelo final que sería presentado a sus clientes, fundamentando el por qué de su elección.
Si durante el desarrollo de este trabajo utilizan y/o aplican más de dos algoritmos pueden compartir la experiencia o resultados (teóricos) de por qué no fueron incluídos en el informe final. Se espera observar en la entrega solo los procedimientos de 2 de los modelos, nada más.